# library

In [616]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import fastai

/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zero/anaconda/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [342]:
from fastai import *

In [627]:
from torch.nn.init import kaiming_normal

In [58]:
from fastai.datasets import *
from fastai.tabular import *
from fastai.basic_data import *

In [68]:
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd

In [205]:
from fastai.basic_train import *

In [297]:
import pdb

In [320]:
from pathlib import Path

In [244]:
from fastai.callbacks import *
from fastai.metrics import *

In [19]:
import numpy as np

In [129]:
from fastai.text import *

In [136]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# read data

In [448]:
#data.to_pickle('telco.pkl')
telco = pd.read_pickle('telco.pkl')

In [449]:
y = telco['churn'].copy()

In [450]:
telco.drop('churn', axis = 1, inplace=True)

In [451]:
x_train, x_val, y_train, y_val = train_test_split(telco, y, test_size=0.2, stratify = y)

## normalize

In [452]:
x_train = (x_train - x_train.mean()) / (x_train.max() - x_train.min())

In [453]:
x_val = (x_val - x_val.mean()) / (x_val.max() - x_val.min())

In [454]:
y_train = (y_train - y_train.mean()) / (y_train.max() - y_train.min())

In [455]:
y_val = (y_val - y_val.mean()) / (y_val.max() - y_val.min())

## sample

In [456]:
x_train = x_train[:100].copy()
y_train = y_train[:100]

In [457]:
x_val = x_val[:100].copy()
y_val = y_val[:100]

## transform

In [458]:
tn = [t for t in telco.columns if t.endswith('tn')]
tn_1 = [t for t in telco.columns if t.endswith('tn_1')]
tn_2 = [t for t in telco.columns if t.endswith('tn_2')]
tn_3 = [t for t in telco.columns if t.endswith('tn_3')]
tn_4 = [t for t in telco.columns if t.endswith('tn_4')]
tn_5 = [t for t in telco.columns if t.endswith('tn_5')]

In [459]:
x_train_behave = np.stack([x_train[t] for t in [tn_5, tn_4, tn_3, tn_2, tn_1]])#.transpose([1,0,2])
x_val_behave = np.stack([x_val[t] for t in [tn_5, tn_4, tn_3, tn_2, tn_1]])#.transpose([1,0,2])
y_train_behave = np.stack([x_train[tn]])[0]
y_val_behave = np.stack([x_val[tn]])[0]

# dataloader

In [389]:
class telcodataset(DatasetBase):
    def __init__(self, x, y):
        self.x = x
        self.y = np.float32(y)
    
    def __len__(self)->int: return len(self.y)
    def __getitem__(self, idx): return self.x[:,idx,:], self.y[idx]
    
    @property
    def c(self)->int: 25

In [390]:
class telcodatabunch(DataBunch):
    "Create a `DataBunch` suitable for tabular data."
    @classmethod
    def from_df(cls, path, x_train, y_train, x_val, y_val, **kwargs):
        "Create a `DataBunch` from train/valid/test dataframes."
        train_ds = telcodataset(x_train, y_train)
        valid_ds = telcodataset(x_val, y_val)
        datasets = [train_ds, valid_ds]
        return cls.create(*datasets, path=path, **kwargs)

In [463]:
db = telcodatabunch.from_df('/', x_train_behave, y_train_behave, x_val_behave, y_val_behave, bs = 16)

## test

In [442]:
a = iter(db.train_dl)

In [443]:
x, y = next(a)

In [314]:
y.shape

torch.Size([16, 25])

In [315]:
type(y)

torch.Tensor

# architechture

## RNN core

In [996]:
class telcoRNNCore(nn.Module):
    "AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182."

    initrange=0.1

    def __init__(self, 
                 #vocab_sz:int, 
                 #emb_sz:int,
                 input_sz:int,
                 n_hid:int, 
                 n_layers:int, 
                 #pad_token:int, 
                 #bidir:bool=False,
                 hidden_p:float=0.2, 
                 input_p:float=0.6, 
                 #embed_p:float=0.1, 
                 weight_p:float=0.5, 
                 #qrnn:bool=False
                ):

        super().__init__()
        self.bs = 1
        self.n_hid,self.n_layers = n_hid,n_layers
        #self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        #self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        #self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        
        self.input_dp = RNNDropout(input_p)
        
        self.rnns = [nn.LSTM(input_sz if l == 0 else n_hid, (n_hid if l != n_layers-1 else input_sz), 1) for l in range(n_layers)]
        #self.rnns = [nn.LSTM(input_sz if l == 0 else n_hid, n_hid, 1) for l in range(n_layers)]
        self.rnns = [WeightDropout(rnn, weight_p) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])
        self.att_seq = attention_seq(ip_sz = 25, op_sz=input_sz,  drop = 0.2) #att_ip_drop
        self.att_fea = attention_fea(ip_sz = 25, op_sz = 25)

    def forward(self, input):
        #sl,bs = input.size()
        #if bs!=self.bs:
        #    self.bs=bs
        #    self.reset()
        raw_output = self.input_dp(input)
        #new_hidden,raw_outputs,outputs = [],[],[]
        raw_outputs,outputs = [],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output.float()) #, self.hidden[l]
            #new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
        
        output = outputs[-1]
        att_seq, att_seq_m = self.att_seq(output, input)
        att_fea, att_fea_m = self.att_fea(output)
        
        #self.hidden = to_detach(new_hidden)
        return output, att_seq, att_seq_m, att_fea, att_fea_m

#     def _one_hidden(self, l:int):
#         "Return one hidden state."
#         nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz)//self.ndir
#         return self.weights.new(self.ndir, self.bs, nh).zero_()

#     def reset(self):
#         "Reset the hidden states."
#         [r.reset() for r in self.rnns if hasattr(r, 'reset')]
#         self.weights = next(self.parameters()).data
#         if self.qrnn: self.hidden = [self._one_hidden(l) for l in range(self.n_layers)]
#         else: self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

## linear decoder

In [997]:
class linear_block(nn.Module):
    def __init__(self, ip_sz, op_sz, drop = None, bias = True, initrange = None, linear = True):
        super().__init__()
        self.linear = linear
        self.op_sz = op_sz
        self.ln = nn.Linear(ip_sz, op_sz, bias = bias)
        kaiming_normal_(self.ln.weight.data)
        if initrange is not None: self.ln.weight.data.uniform_(-initrange, initrange)
        if bias: self.ln.bias.data.zero_()
        if drop is not None:
            self.bn = nn.BatchNorm1d(op_sz)
            self.drp = nn.Dropout(drop)
        
    def forward(self, x):
        x = x.float()
        x = self.ln(x) if self.linear else F.relu(self.drp(self.bn(self.ln(x))))
        return x

In [998]:
class LinearDecoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."

    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, output_p:float, bias:bool=True):
        super().__init__()
        self.concat_lyrs = linear_block(175, 32, 0.2 , linear= False) #concat_drop
        
        self.decoder = linear_block(32, 25, drop = 0.2, initrange = self.initrange, bias=bias)
        #self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        #if bias: self.decoder.bias.data.zero_()
        #if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input):
        output, att_seq, att_seq_m, att_fea, att_fea_m = input
        output = self.output_dp(output)
        output = output.view(output.size(0), -1)
        x = self.concat_lyrs(torch.cat([output, att_seq, att_fea], 1))        
        #decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        decoded = self.decoder(x)
        return decoded.float()

In [999]:
class PoolingLinearClassifier(nn.Module):
    "Create a linear classifier with pooling."

    def __init__(self, output_p):
        super().__init__()
        self.concat_lyrs = linear_block(250, 32, 0.2 , linear= False) #concat_drop
        self.decoder = linear_block(32, 1, drop = 0.2)
        self.output_dp = RNNDropout(output_p)

    def pool(self, x:Tensor, bs:int, is_max:bool):
        "Pool the tensor along the seq_len dimension."
        f = F.adaptive_max_pool1d if is_max else F.adaptive_avg_pool1d
        #return f(x.permute(1,2,0), (1,)).view(bs,-1)
        return f(x.permute(0,2,1), (1,)).view(bs,-1)

    def forward(self, input):
        output, att_seq, att_seq_m, att_fea, att_fea_m = input
        bs = output.size(0)
        avgpool = self.pool(output, bs, False)
        mxpool = self.pool(output, bs, True)
        
        output = self.output_dp(output)
        output = output.view(output.size(0), -1)        

        
        x = self.concat_lyrs(torch.cat([output, att_seq, att_fea, avgpool, mxpool], 1))        
        #decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        decoded = self.decoder(x).float()
        
        return decoded.view(-1), att_seq_m, att_fea_m

## attention

In [987]:
def rand_p(*sz): return nn.Parameter(torch.randn(sz)/math.sqrt(sz[0]))

In [995]:
class attention_fea(nn.Module):
    def __init__(self, ip_sz = 64, op_sz = 36, drop = 0.2):
        super().__init__()
        self.w = rand_p(ip_sz, op_sz)
        self.ln = linear_block(op_sz, op_sz, drop)
        self.v = rand_p(op_sz, op_sz)
             
    def forward(self, outp):
        we = outp@self.w
        u = torch.tanh(we)
        at = F.softmax((u@self.v).sum(1), 1)
        xa = self.ln(at) #.double()
        return xa, at

In [989]:
class attention_seq(nn.Module):
    def __init__(self, ip_sz = 64, op_sz = 36, drop = 0.2):
        super().__init__()
        self.w = rand_p(ip_sz, op_sz)
        self.ln = linear_block(op_sz, op_sz, drop)
        self.v = rand_p(op_sz, op_sz)
              
    def forward(self, outp, inp):
        we = outp@self.w
        u = torch.tanh(we)
        at = F.softmax(u@self.v, 1)
        xa = self.ln((at.double()*inp).sum(1))
        return xa, at

# behavior learner

In [812]:
def telco_lm_split(model):
    "Split a RNN `model` in groups for differential learning rates."
    groups = [[rnn, dp] for rnn, dp in zip(model[0].rnns, model[0].hidden_dps)]
    groups.append([model[0].input_dp, model[1]])
    return groups

In [833]:
def rnn_classifier_split(model):
    "Split a RNN `model` in groups for differential learning rates."
    groups = [[rnn, dp] for rnn, dp in zip(model[0].rnns, model[0].hidden_dps)]
    groups.append([model[1]])
    return groups

In [835]:
class RNNLearner(Learner):
    "Basic class for a Learner in RNN."
    def __init__(self,
                 path,
                 data:DataBunch, 
                 model, 
                 #bptt:int=70, 
                 split_func=None, 
                 clip:float=None,
                 adjust:bool=False, 
                 alpha:float=2., 
                 beta:float=1., 
                 **kwargs):
        super().__init__(data, model, path = path, model_dir = 'models', **kwargs)
        self.path = path
        #self.callbacks.append(RNNTrainer(self, bptt, alpha=alpha, beta=beta, adjust=adjust))
        if clip: self.callback_fns.append(partial(GradientClipping, clip=clip))
        if split_func: self.split(split_func)
        self.metrics = None #[accuracy]

    def save_encoder(self, name:str):
        "Save the encoder to `name` inside the model directory."
        torch.save(self.model[0].state_dict(), self.path/f'{name}.pth')

    def load_encoder(self, name:str):
        "Load the encoder `name` from the model directory."
        self.model[0].load_state_dict(torch.load(self.path/f'{name}.pth'))
        self.freeze()

    @classmethod
    def language_model(cls,
                       path,
                       data:DataBunch, 
                       #bptt:int=70, 
                       #emb_sz:int=400,
                       input_sz = 25,
                       nh:int=1150, 
                       nl:int=3, 
                       #pad_token:int=1,
                       drop_mult:float=1., 
                       #tie_weights:bool=True, 
                       bias:bool=True, 
                       #qrnn:bool=False, 
                       #pretrained_model=None,
                       #pretrained_fnames:OptStrTuple=None, 
                       **kwargs):
        "Create a `Learner` with a language model."
        dps = np.array([0.25, 0.1, 0.2, 0.02, 0.15]) * drop_mult
        
        rnn_enc = telcoRNNCore(input_sz = input_sz, n_hid=nh, n_layers=nl, input_p=dps[0], weight_p=dps[2], hidden_p=dps[4])
        #lnr_dec = LinearDecoder(n_out = input_sz, n_hid = input_sz, output_p= dps[1], bias= bias)
        lnr_dec = LinearDecoder(n_out = input_sz, n_hid = input_sz*5, output_p= dps[1], bias= bias)
        
        #vocab_size = data.train_ds.vocab_size
        model = SequentialRNN(rnn_enc, lnr_dec)
        learn = cls(path, data, model, split_func=telco_lm_split, **kwargs) #bptt,
        return learn
    
    @classmethod
    def classifier(cls,
                   path,
                   data,
                   input_sz = 25,
                   #bptt:int=70, 
                   #max_len:int=70*20, 
                   #emb_sz:int=400, 
                   nh:int=1150, 
                   nl:int=3,
                   #lin_ftrs:Collection[int]=None, 
                   ps:Collection[float]=None, 
                   #pad_token:int=1,
                   drop_mult:float=1., 
                   #qrnn:bool=False, 
                   **kwargs):
        "Create a RNN classifier."
        dps = np.array([0.4,0.5,0.05,0.3,0.4]) * drop_mult
        #if lin_ftrs is None: lin_ftrs = [50]
        if ps is None:  ps = [0.1]
        rnn_enc = telcoRNNCore(input_sz = input_sz, n_hid=nh, n_layers=nl, input_p=dps[0], weight_p=dps[2], hidden_p=dps[4])
        #lnr_dec = LinearDecoder(n_out = input_sz, n_hid = input_sz, output_p= dps[1], bias= bias)
        lnr_dec = PoolingLinearClassifier(output_p= dps[1])
        model = SequentialRNN(rnn_enc, lnr_dec)
        learn = cls(path, data, model, split_func=rnn_classifier_split, **kwargs) #bptt,
        return learn

In [825]:
learner = RNNLearner.language_model(Path('./models'), db, input_sz = 25, nh = 8, nl = 3, drop_mult = 0.5 )

In [816]:
def loss(input, target):
    #pdb.set_trace()
    input, att_seq_m, att_fea_m = input
    input = input.float()
    target = target.float()
    return F.mse_loss(input, target, reduction='sum')

In [817]:
learner.loss_func = loss

In [790]:
learner.metrics = None

In [791]:
learner.fit_one_cycle(4)

VBox(children=(HBox(children=(IntProgress(value=0, max=4), HTML(value='0.00% [0/4 00:00<00:00]'))), HTML(value='epoch  train loss  valid loss<p>')))

RuntimeError: size mismatch, m1: [80 x 8], m2: [25 x 25] at /opt/conda/conda-bld/pytorch-nightly-cpu_1538636779972/work/aten/src/TH/generic/THTensorMath.cpp:932

In [779]:
learner.save_encoder('behavior_model')

# classifier

## dataset

In [977]:
x_train_class = np.stack([x_train[t] for t in [tn_5, tn_4, tn_3, tn_2, tn_1, tn]])#.transpose([1,0,2])
x_val_class = np.stack([x_val[t] for t in [tn_5, tn_4, tn_3, tn_2, tn_1, tn]])#.transpose([1,0,2])
y_train_class = np.array(y_train)
y_val_class = np.array(y_val)

In [978]:
db = telcodatabunch.from_df('/', x_train_class, y_train_class, x_val_class, y_val_class, bs = 16)

## learner

In [1000]:
learner = RNNLearner.classifier(Path('./models'), db, input_sz = 25, nh = 8, nl = 3, drop_mult = 0.5 )

In [1001]:
def loss(input, target):
    input, att_seq_m, att_fea_m = input
    #pdb.set_trace()
    input = input.float()
    target = target.float()
    return F.binary_cross_entropy_with_logits(input, target, reduction='sum')

In [1002]:
learner.loss_func = loss

In [1003]:
learner.metrics = None

In [1004]:
learner.fit_one_cycle(5)

VBox(children=(HBox(children=(IntProgress(value=0, max=5), HTML(value='0.00% [0/5 00:00<00:00]'))), HTML(value='epoch  train loss  valid loss<p>')))

Total time: 00:05
epoch  train loss  valid loss
1      10.872232   22.735891   (00:01)
2      10.195671   21.863991   (00:01)
3      9.480897    21.200529   (00:01)
4      8.851356    20.401417   (00:01)
5      8.474024    18.498566   (00:01)



# permutation importance

# visualization